In [1]:
import pickle
import pandas as pd

In [2]:
x = open("political_tweets.csv", 'r')

In [96]:
i = 3
for line in x:
    if i < 10:
        print(line)
        i = i+1

0,2016-02-28 23:07:44,Go Bernie!  #FeelTheBern  #SuperTuesday Democracy is not a spectator sport! GET OUT AND VOTE! #Oscars https://t.co/MntIAPCiKl

1,2016-02-28 23:07:44,.@freep endorses @HillaryClinton. Calls her the best choice for MI Democrats: https://t.co/6dDZjNSPTt

2,2016-02-28 23:07:45,"State Dept. Releases 1,500 Clinton Emails https://t.co/ZXazBTNDAo #Newsmax via Newsmax_Media"

3,2016-02-28 23:07:45,#ReleaseTheTranscripts so we can see #WhichHillary https://t.co/XdmAKxwZ9C

4,2016-02-28 23:07:45,Last year Bernie Sanders told NPR black people couldn't comprehend his policies. Now people of color in  #NV &amp; #SC are less than smart.

5,2016-02-28 23:07:45,"Wonder if Bernie even knows? #ImWithHer The story is about dirty, dirty politics against Hillary #HRC  https://t.co/DFUjuC069I"

6,2016-02-28 23:07:46,Hillary waits to eviscerate Trump with a parade of his victims https://t.co/ayp5OKnZL7 via @DCExaminer



In [3]:
x = open("political_tweets.csv", 'r')
date = []
text = []
shit = []
for line in x:
    row = line.split(',')
    try:       
        text.append(row[2])
        date.append(row[1])
    except:
        shit.append(row)

In [4]:
df = pd.DataFrame([date,text]).T

In [97]:
lemmatize('fishery')

'fishery'

In [121]:
import string
from nltk.stem import WordNetLemmatizer
from functools import lru_cache

wnl = WordNetLemmatizer()
lemmatize = lru_cache(maxsize=50000)(wnl.lemmatize)
lemmatize('dogs')

def lemmas(tweet):
    return ' '.join([lemmatize(word) for word in tweet.split(" ")])


def collect(tweet,char):
    """Input: tweet and Output: hashtags"""
    collection = []
    for word in tweet.split(" "):
        if len(word) > 0 and word[0:len(char)] == char:
            collection.append(word[len(char):len(word)].rstrip('\n'))
    return collection

collect = lru_cache(maxsize=50000)(collect)


def remove_char(tweet,char):
    """Input: tweet and Output: tweet without specified char"""
    words = [word for word in tweet.split(" ") if len(word) > 0]
    return ' '.join([word for word in words if word[0:len(char)] != char])

remove_char = lru_cache(maxsize=50000)(remove_char)

def count_chars(tweet,char):
    """Input: tweet and Output: char count"""
    return len([letter for letter in tweet if letter == char])

count_chars = lru_cache(maxsize=50000)(count_chars)

def remove_punct(tweet):
    """Input: tweet and Output: tweet without specified char"""

    exclude = string.punctuation #set([',','.','!',';',":","?",""])
    return ''.join(ch for ch in tweet if ch not in exclude)

remove_punct = lru_cache(maxsize=50000)(remove_punct)

def remove_chars(tweet,char_list):
    """Input: tweet and Output: tweet without specified char"""
    exclude = char_list
    return ''.join(ch for ch in tweet if ch not in exclude)


def lowercase(tweet):
    """Input: tweet and Output: tweet without specified char"""
    return tweet.lower()
lowercase = lru_cache(maxsize=50000)(lowercase)

def remove_numbers(tweet):
    return ''.join(ch for ch in tweet if not ch.isdigit())

remove_numbers = lru_cache(maxsize=50000)(remove_numbers)
# def expand_contractions(tweet):
#     """Input: tweet and Output: tweet with expanded contractions"""
#     contractions[ch]
#     ''.join(ch for ch in tweet if ch not in exclude)
# # def 

In [111]:
lemmas(df[1].iloc[0])

['.@freep',
 'endorses',
 '@HillaryClinton.',
 'Calls',
 'her',
 'the',
 'best',
 'choice',
 'for',
 'MI',
 'Democrats:',
 'https://t.co/6dDZjNSPTt\n']

In [117]:
def clean_tweet(tweet):
    new_tweet = remove_punct(tweet)
    new_tweet = remove_chars(new_tweet,['#','@','\n','&',"\""])
    new_tweet = remove_char(new_tweet,'https')
    new_tweet = lowercase(new_tweet)
    new_tweet = remove_numbers(new_tweet)
    new_tweet = lemmas(new_tweet)
    return new_tweet

In [85]:
we = '12 12'+ df[1].iloc[0]+' 11'
remove_numbers(we)

' .@freep endorses @HillaryClinton. Calls her the best choice for MI Democrats: https://t.co/dDZjNSPTt\n '

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
# from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn.metrics import accuracy_score
# from nltk.corpus import stopwords
import string
import re

In [41]:
# A custom stoplist
STOPLIST = set(stopwords.words('english') + ["n't", "'s", "'m", "ca"] + list(ENGLISH_STOP_WORDS))
# List of symbols we don't care about
SYMBOLS = " ".join(string.punctuation).split(" ") + ["-----", "---", "...", "“", "”", "'ve"]

In [163]:
# Every step in a pipeline needs to be a "transformer". 
# Define a custom transformer to clean text using spaCy
class CleanTextTransformer(TransformerMixin):
    """
    Convert text to cleaned text
    """

    def transform(self, X, **transform_params):
        return [cleanText(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}
    
# A custom function to clean the text before sending it into the vectorizer
def cleanText(text):
    # get rid of newlines
    text = text.strip().replace("\n", " ").replace("\r", " ")

    # lowercase
    text = text.lower()

    return text

# A custom function to tokenize the text using spaCy
# and convert to lemmas
def tokenizeText(sample):
    
    # get the tokens using spaCy
    tokens = parser(sample)

    # lemmatize
    lemmas = []
    for tok in tokens:
        lemmas.append(tok.lemma_.lower().strip() if tok.lemma_ != "-PRON-" else tok.lower_)
    tokens = lemmas

    # stoplist the tokens
    tokens = [tok for tok in tokens if tok not in STOPLIST]

    # stoplist symbols
    tokens = [tok for tok in tokens if tok not in SYMBOLS]

    # remove large strings of whitespace
    while "" in tokens:
        tokens.remove("")
    while " " in tokens:
        tokens.remove(" ")
    while "\n" in tokens:
        tokens.remove("\n")
    while "\n\n" in tokens:
        tokens.remove("\n\n")

    return tokens

tokenizeText = lru_cache(maxsize=50000)(tokenizeText)


def printNMostInformative(vectorizer, clf, N):
    """Prints features with the highest coefficient values, per class"""
    feature_names = vectorizer.get_feature_names()
    coefs_with_fns = sorted(zip(clf.coef_[0], feature_names))
    topClass1 = coefs_with_fns[:N]
    topClass2 = coefs_with_fns[:-(N + 1):-1]
    print("Class 1 best: ")
    for feat in topClass1:
        print(feat)
    print("Class 2 best: ")
    for feat in topClass2:
        print(feat)
  

In [122]:
df['clean'] = df[1].apply(lambda tweet: clean_tweet(tweet))

In [123]:
df = df.iloc[1::]

In [124]:
df.head()

,0,1,clean
5,2016-02-28 23:07:45,Last year Bernie Sanders told NPR black people...,last year bernie sander told npr black people ...
6,2016-02-28 23:07:45,"""Wonder if Bernie even knows? #ImWithHer The s...",wonder if bernie even know imwithher the story...
7,2016-02-28 23:07:46,Hillary waits to eviscerate Trump with a parad...,hillary wait to eviscerate trump with a parade...
8,2016-02-28 23:07:46,@SusanSarandon @HillaryClinton Goodnight Hill...,susansarandon hillaryclinton goodnight hillary
9,2016-02-28 23:07:47,"""Clinton allies preparing for Trump nomination",clinton ally preparing for trump nomination


In [154]:
ats = df[1].apply(lambda tweet: collect(tweet,'@'))
hashtag = df[1].apply(lambda tweet: collect(tweet,'#'))
urls = df[1].apply(lambda tweet: collect(tweet,'https://'))
exclams = df[1].apply(lambda tweet: count_chars(tweet,'!'))

In [155]:
df["ats"],df["hashtag"],df["urls"],df["exclams"] = ats,hashtag,urls,exclams

In [10]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [11]:
from gensim import corpora, models, similarities

In [155]:
frac = df.sample(frac=0.1)

documents = list(df.clean)
frac_doc = list(frac.clean)

In [126]:
len(documents)

309706

In [12]:
stoplist = set('for a of the and to in'.split())
texts = [[word for word in document.lower().split() if word not in stoplist]
      for document in documents]

# remove words that appear only once
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
         frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1]
      for text in texts]

from pprint import pprint   # pre

In [13]:
dictionary = corpora.Dictionary(texts)

In [14]:
dictionary.save('/tmp/hillary.dict')

In [20]:
print(dictionary)
type(dictionary)

Dictionary(47892 unique tokens: ['ahead…', 'ferdddaws', 'win/lose', 'oligarchs4hillary', 'bloglovin']...)


gensim.corpora.dictionary.Dictionary

In [23]:
# pd.DataFrame.from_csv("political_tweets.csv")

from sklearn.feature_extraction.text import sklearn 
from sklearn.metrics.pairwise import pairwise_distances

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(list(df[1]))


In [156]:
len(frac_doc)

30971

In [158]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(encoding='utf-8', decode_error='strict', 
                             strip_accents=None, lowercase=True, preprocessor=None, 
                             tokenizer=tokenizeText, stop_words=STOPLIST, token_pattern='(?u)\b\w\w+\b', 
                             ngram_range=(1, 1), analyzer='word', max_df=1000, min_df=3, 
                             max_features=None)

In [159]:
X = vectorizer.fit_transform(frac_doc)

In [160]:
words = vectorizer.get_feature_names()

In [162]:
words

['--',
 'aa',
 'aaron',
 'aarp',
 'ab',
 'abandon',
 'abc',
 'abcnews',
 'abcpolitics',
 'abedin',
 'ability',
 'able',
 'aboard',
 'abolish',
 'abortion',
 'abowersock',
 'abroad',
 'absent',
 'absolute',
 'absolutely',
 'absurd',
 'abt',
 'abuse',
 'abusive',
 'ac',
 'aca',
 'academy',
 'accent',
 'accept',
 'acceptance',
 'access',
 'accident',
 'accidentally',
 'accomplish',
 'accomplishment',
 'accord',
 'account',
 'accountable',
 'accurate',
 'accurately',
 'accusation',
 'accuse',
 'accuser',
 'ace',
 'achievement',
 'acknowledge',
 'aclu',
 'act',
 'actblue',
 'action',
 'actively',
 'activist',
 'activity',
 'actor',
 'actress',
 'actual',
 'actually',
 'ad',
 'adam',
 'adamsflafan',
 'adapt',
 'add',
 'address',
 'adele',
 'admin',
 'administration',
 'admire',
 'admirer',
 'admit',
 'adopt',
 'adorable',
 'adult',
 'advance',
 'advantage',
 'advert',
 'advice',
 'advise',
 'adviser',
 'advisor',
 'advocate',
 'af',
 'affect',
 'affiliation',
 'afford',
 'affordable',
 'afgh

In [143]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [30]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords
import string
import re

In [32]:
from spacy.en import English
parser = English()

In [ ]:
from nltk.stem import WordNetLemmatizer
from functools import lru_cache

wnl = WordNetLemmatizer()
lemmatize = lru_cache(maxsize=50000)(wnl.lemmatize)
lemmatize('dogs')

In [35]:
wnl = WordNetLemmatizer()
lemmatize = lru_cache(maxsize=50000)(wnl.lemmatize)
lemmatize('dogs')

'dog'

In [38]:
messyData = "lol that is rly funny :) This is gr8 i rate it 8/8!!!"
parsedData = parser(messyData)
for token in parsedData:
    print(token.orth_, token.pos_, token.lemma_)

lol NOUN lol
that ADJ that
is VERB be
rly ADV rly
funny ADJ funny
:) PUNCT :)
This DET this
is VERB be
gr8 VERB gr8
i NOUN i
rate VERB rate
it NOUN it
8/8 NUM 8/8
! PUNCT !
! PUNCT !
! PUNCT !


In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer()

In [ ]:
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
y = np.array([1, 1, 1, 2, 2, 2])
clf = LinearDiscriminantAnalysis()
clf.fit(X, y)
LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
              solver='svd', store_covariance=False, tol=0.0001)
print(clf.predict([[-0.8, -1]]))
[1]